# <span style="color:red">  **SIMULATIONS ET MÉTHODES DE MONTE CARLO**  </span>

In [1]:
import numpy as np 
from scipy.stats import multivariate_normal
import Estimateurs
import Code
import matplotlib.pyplot as plt
from tqdm import tqdm

<span style="color:blue">  **NOTATIONS**  </span> : on se donne un ensemble de données $\boldsymbol{x} = \{x^{(i)}\}_{i=1}^n$ composé de $n$ échantillons i.i.d. d'une variable continue ou discrète $x$ à valeurs dans un espace d'observations $\boldsymbol{\mathcal{X}}$ (ainsi $\boldsymbol{x} \in \boldsymbol{\mathcal{X}}$). Nous supposons que les données sont générées par un processus aléatoire impliquant une variable aléatoire continue non observée $\boldsymbol{z} = \{z^{(i)}\}_{i=1}^n$ à valeur dans un espace d'observations $\boldsymbol{\mathcal{Z}}$ (ainsi $\boldsymbol{z} \in \boldsymbol{\mathcal{Z}}$). Le processus se compose de deux étapes :


1. La valeur $z^{(i)}$ est générée à partir d'une distribution a priori $p_{\theta^*}(z)$ ;
2. Une valeur $x^{(i)}$ est générée à partir d'une distribution conditionnelle $p_{\theta^*}(x|z)$.


Nous supposons que la distribution a priori $p_{\theta^*}(\boldsymbol{z})$ et la vraisemblance $p_{\theta^*}(\boldsymbol{x}|\boldsymbol{z})$ proviennent de familles paramétriques de distributions $p_{\theta}(\boldsymbol{z})$ et $p_{\theta}(\boldsymbol{x}|\boldsymbol{z})$ et on note $\Theta \subset \mathbb{R}^d$ l'espace des paramètres. On connait ainsi l'expression de ces deux distributions. En revanche une grande partie de ce processus nous est cachée : les véritables paramètres $\theta^*$ ainsi que les valeurs des $\textbf{variables latentes}$ $z^{(i)}$ nous sont inconnus. 

Connaissant l'expression de la distribution a priori $p_{\theta^*}(\boldsymbol{z})$ et la vraisemblance $p_{\theta^*}(\boldsymbol{x}|\boldsymbol{z})$ pour tout $\theta \in \Theta$, on peut définir la densité jointe $p_{\theta}(\boldsymbol{x}, \boldsymbol{z})$ sur $\boldsymbol{\mathcal{X}} \times \boldsymbol{\mathcal{Z}}$ par : 

$$
p_{\theta}(\boldsymbol{x}, \boldsymbol{z}) = p_{\theta}(\boldsymbol{x}|\boldsymbol{z}) p_{\theta}(\boldsymbol{z})
$$

<span style="color:blue">  **OBJECTIF**  </span> : on cherche à déterminer la vraie valeur du paramètre (ici $\theta^*$) ainsi que les valeurs des variables latentes. 

Une approche classique pour apprendre $\boldsymbol{\theta}$ est de choisir, si celle-ci existe, la valeur de ce paramètre qui maximise la log-vraisemblance marginale de l’ ́echantillon définie par :

$$
\ell(\boldsymbol{\theta}) = \log p_{\boldsymbol{\theta}}(\boldsymbol{x}) = \log \int_{\boldsymbol{\mathcal{Z}}} p_{\boldsymbol{\theta}}(\boldsymbol{x}, \boldsymbol{z}) d\boldsymbol{z}
$$

<span style="color:blue">  **PROBLÈME**  </span> : l'intégrale donnée en ci-dessus est intractable (nous ne pouvons donc pas évaluer ou différencier la vraisemblance marginale).

<span style="color:blue">  **SOLUTION**  </span> : dans le but de résoudre les problèmes ci-dessus, introduisons un **modèle de reconnaissance** (paramétrique) {$ q_{\phi}(\boldsymbol{z}|\boldsymbol{x}) : \phi \in \Phi$} avec $\Phi \subset \mathbb{R}^d$. Pour tout $\phi$, $q_{\phi}(\boldsymbol{z}|\boldsymbol{x})$ est choisi comme une approximation de la véritable postérieure intractable $p_{\theta}(\boldsymbol{z}|\boldsymbol{x})$. L'idée est donc de proposer une valeur de l'a posteriori (faire une hypothèse) et d'introduire une méthode pour apprendre les paramètres du **modèle de reconnaissance** $\phi$ conjointement avec les paramètres du **modèle génératif** $\theta$.

## <span style="color:green">  **Partie 0. Définition du cadre gaussien pour la génération des données**  </span>

(Application numérique inspirée de [Rainforth et al. (2018)](https://arxiv.org/pdf/1802.04537.pdf))

- Le **modèle génératif** est donné par $p_{\theta}(\boldsymbol{x}, \boldsymbol{z}) = \mathcal{N}(\boldsymbol{z}|\theta, I) \mathcal{N}(\boldsymbol{x}|\boldsymbol{z}, I)$, où $\boldsymbol{x}$ et $\boldsymbol{z} \in \mathbb{R}^{20}$, de sorte que $p_{\theta}(\boldsymbol{x}) = \mathcal{N}(\boldsymbol{x}|\theta, 2I)$ et $p_{\theta}(\boldsymbol{z}|\boldsymbol{x}) = \mathcal{N}\left( \frac{\theta + x}{2}, \frac{1}{2}I \right)$. 

- La **distribution de l'encodeur** (le modèle de reconnaissance) est $q_{\phi}(z|x) = \mathcal{N}\left(\boldsymbol{z}|A\boldsymbol{x} + b, \frac{2}{3}I \right)$, où $\phi = (A, b)$.

- Nous considérons des perturbations aléatoires des paramètres près de la valeur optimale par une distribution gaussienne de moyenne nulle et d'écart-type $0.01$.

**Dans ce cas, on peut analytiquement calculer la vraie log-vraisemblance du modèle pour quantifier le biais et la variance de tous les estimateurs**.

Puisque $q_{\phi}(z|x) = \mathcal{N}\left(\boldsymbol{z}|A\boldsymbol{x} + b, \frac{2}{3}I \right)$ est censé être une approximation de la véritable loi a posteriori donnée par $p_{\boldsymbol{\theta}}(\boldsymbol{z} | \boldsymbol{x}) = \mathcal{N}\left( \frac{\theta + x}{2}, \frac{1}{2}I \right)$

#### <span style="color:orange"> 0.1. **Génération du vrai $\theta$**  </span>

On commence par tirer le $\theta$* ($\in \mathbb{R}$) qui sera le paramètre que l'on cherchera à estimer par la suite. 

A la manière de la génération des données dans [Rainforth et al. (2018)](https://arxiv.org/pdf/1802.04537.pdf) : $\theta^* \sim \mathcal{N}(0, 1)$

In [2]:
#theta_true = np.random.multivariate_normal(np.zeros(20), np.identity(20))
theta_true = np.random.normal(0, 1)
print(f"La valeur de theta à estimer est {int(theta_true*100)/100}")

La valeur de theta à estimer est -0.54


#### <span style="color:orange"> 0.2. **Génération de notre vecteur d'observation $\boldsymbol{x} \in \mathbb{R}^{20}$**  </span>

L'idée est désormais de tirer une observation $\boldsymbol{x} \in \mathbb{R}^{20}$, chose que l'on peut faire puisque toutes les distributions nous sont données. 


In [3]:
## On se donne notre échantillon x

x, _ = Estimateurs.joint_probability(theta_true)

print(f"L'échantillon x observé est :\n \n x = {x} \n \n et on vérifique bien sa taille est celle voulue : {x.shape}")

L'échantillon x observé est :
 
 x = [ 0.97338888 -2.73170457 -1.6806929   1.05083576 -2.21507264 -3.18502551
 -2.23967998 -0.45062161 -1.76321727  3.15293986  0.40634669 -0.67576346
 -0.70036704  2.22620484 -3.09627883 -0.47438378 -2.50192095 -1.12050425
 -0.64540774 -2.3607701 ] 
 
 et on vérifique bien sa taille est celle voulue : (20,)


#### <span style="color:orange"> 0.3. **Génération de la vraie log-vraisemblance et du vrai gradient associé à $\theta_{\text{true}}$ et à notre observation $\boldsymbol{x}$**  </span>

In [4]:
true_likelihood = Code.true_likelihood(x, theta_true)
true_gradient = Code.true_grad(x, theta_true)

print(f"La valeur associée à la log-vraisemblance de l'échantillon x pour theta_true est : {true_likelihood} \n ")
print(f"La valeur associée au gradient de l'échantillon x pour theta_true est : \n\n {true_gradient} \n ")

La valeur associée à la log-vraisemblance de l'échantillon x pour theta_true est : -34.070631965652126 
 
La valeur associée au gradient de l'échantillon x pour theta_true est : 

 [-0.75740826  1.09513846  0.56963263 -0.7961317   0.83682249  1.32179893
  0.84912617 -0.04540302  0.61089481 -1.84718375 -0.47388717  0.06716791
  0.07946969 -1.38381624  1.27742559 -0.03352193  0.98024665  0.2895383
  0.05199004  0.90967123] 
 


#### <span style="color:orange"> 0.4. **Génération des paramètres de l'encodeur $A \in \mathbb{R}^{20\times 20}$ et $b \in \mathbb{R}^{20}$**  </span>

Il est maintenant question de tirer notre encodeur tel que : $q_{\phi}(z|x) = \mathcal{N}\left(\boldsymbol{z}|A\boldsymbol{x} + b, \frac{2}{3}I \right)$, où $\phi = (A, b)$. **Le problème qui se pose est donc celui du choix de $A$ et de $b$**. Dans l'article [Rainforth et al. (2018)](https://arxiv.org/pdf/1802.04537.pdf), il nous est indiqué : 

*"Following [Rainforth et al. (2018)](https://arxiv.org/pdf/1802.04537.pdf), we consider random perturbations of the parameters near the optimal value by a zero-mean Gaussian with standard deviation 0.01"*

Ainsi, étant donné une observation $\boldsymbol{x}$,  le choix parfait pour $\phi$ serait $\phi^* = (A^*, b^*)$ où $A^* = \frac{1}{2}\boldsymbol{I}_{20} \in \mathbb{R}^{20 \times 20}$ et $b^* = [\frac{\theta^*}{2}, ..., \frac{\theta^*}{2}]^T \in \mathbb{R}^{20} $. Suivant l'indication dans l'article, nous nous plaçons dans un cas où on aurait réussi à inférer de manière convenable la loi $p_{\theta}(\boldsymbol{z}|\boldsymbol{x}) = \mathcal{N}\left( \frac{\theta + x}{2}, \frac{1}{2}I \right)$, mais pas parfaitement. Ainsi,  on introduit de la même façon que dans l'article [[Rainforth et al. (2018)](https://arxiv.org/pdf/1802.04537.pdf) une perturbation autour de la valeur optimale du paramètre $\phi$. Cette perturbation se caractérise par le fait chaque dimension de chaque paramètre a été décalée de manière aléatoire par rapport à sa valeur optimale en utilisant une gaussienne centrée en zéro avec un écart-type de 0,01.


In [5]:
dim = 20 

## On calcule les valeurs optimales de A et de b
A = 0.5 * np.eye(dim)
b = 0.5 * theta_true * np.ones(dim)

## On calcule les valeurs perturbées de A et de b, qui sont FIXÉES dans la suite
noised_A, noised_b = Code.noised_params(A, b)

print(f"On vérifie que les tailles de A, b et x coincident :\n \n taille A : {A.shape} \n taille b : {b.shape} \n taille x : {x.shape}")

On vérifie que les tailles de A, b et x coincident :
 
 taille A : (20, 20) 
 taille b : (20,) 
 taille x : (20,)


## <span style="color:green">  **Partie 1. L'estimateur SUMO (Stochastically Unbiased Marginalization Objective (SUMO))**  </span>

### <span style="color:orange">  **1.1 Les estimateurs RR (Roulette Russe) et SS (Single Sample)**  </span>

On rappelle rapidement la manière dont sont définis les estimateurs RR et SS, utilisés plus tard pour construire SUMO, ML-SS et ML-RR.  
Désignons une quantité d'intérêt par $I_\infty = \log p_\theta(\boldsymbol{x})$. Supposons que $I_\infty$ puisse être écrite comme

$$
I_\infty = \mathbb{E}[I_0] + \sum_{k=0}^\infty \mathbb{E}[\Delta_k] 
$$

pour les variables aléatoires $I_0$ et $(\Delta_k)_{k\geq0}$. Nous pouvons estimer $I_\infty$ de manière non biaisée via les estimateurs suivants ss ou rr :

$$
ss = I_0 + \frac{\Delta_K}{p(K)}, \quad rr = I_0 + \sum_{k=0}^K \frac{\Delta_k}{\mathbb{P}(K \geq k)}
$$

où $K \sim Geom(r)$.

### <span style="color:orange">  **1.2 L'estimateurs SUMO (Stochastically Unbiased Marginalization Objective (SUMO))**  </span>

On pose 
\begin{align*}
\Delta^{\text{SUMO}}_k &:= \hat{\ell}^{(k+2)}(\boldsymbol{\theta}) - \hat{\ell}^{(k+1)}(\boldsymbol{\theta})\\
&:= \log \left( \frac{1}{k+2} \sum_{i=1}^{k+2} w(\boldsymbol{z}_i) \right) - \log \left( \frac{1}{k+1} \sum_{i=1}^{k+1} w(\boldsymbol{z}_i) \right),
\end{align*}

où on rappelle que $w(\boldsymbol{z}) := \frac{p_{\boldsymbol{\theta}}(\boldsymbol{x}, \boldsymbol{z})}{q_{\phi}(\boldsymbol{z}|\boldsymbol{x})}$.

On applique l'estimateur $RR$ pour construire notre estimateur, que nous appelons $\textbf{SUMO}$ (Stochastically Unbiased Marginalization Objective) qui correspond précisément à l'estimateur de la roulette russe pour $\Delta^{\text{SUMO}}_k$. Ainsi, 
$$
\hat{\ell}^{\text{SUMO}}(\boldsymbol{\theta}) := I_0 + \sum^K_{k=0} \frac{\Delta^{\text{SUMO}}_k}{P(\mathcal{K} \geq k)}
$$
    
où $\mathcal{K} \sim p(\cdot)$ une distrubtion de support dans $\mathbb{N}$

In [6]:
n_simulations = 10 
r = 0.6

theta_min = theta_true - 5  # Limite inférieure de la plage
theta_max = theta_true + 5 # Limite supérieure de la plage
num_points = 30  # Nombre de points à générer
theta_values = np.linspace(theta_min, theta_max, num_points)

estimated_likelihood = []

with tqdm(total=n_simulations) as pbar:

    for theta in theta_values:

        estimated_likelihood.append(Code.log_likelihood_SUMO(r, theta, x, noised_A, noised_b, n_simulations))

        pbar.update(1)


true_likelihood_values = [true_likelihood(x, theta) for theta in theta_values]

plt.plot(theta_values, true_likelihood_values, color='r', label='True likelihood')  
plt.scatter(theta_values, estimated_likelihood, color='purple', marker='x', label='SUMO')
plt.axvline(x=theta_true, color='black', linestyle='--', label='theta='+ str('{:.2f}'.format(theta_true)))
plt.ylim([-300,500])
plt.xlabel('Theta')
plt.ylabel('Likelihood')
plt.title('Estimation de la likelihood par SUMO')
plt.legend(loc='best')
plt.show()

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]


TypeError: unsupported operand type(s) for /: 'generator' and 'int'